<a href="https://colab.research.google.com/github/Viktory4121/MORD/blob/main/%D0%94%D0%9E%D0%9C.ipynb" target="_parent"><img src="https://colab.research.google.com/assets/colab-badge.svg" alt="Open In Colab"/></a>

#Динамические окрестностные модели (ДОМ)
####Вариант 5

In [1]:
import numpy as np
import pandas as pd
import random
import copy
import math
from prettytable import PrettyTable
import itertools as it

In [ ]:
#A={a1, a2, a3}

#какие дуги входят в позицию:
#Ox[a1]={a1}
#Ox[a2]={}
#Ox[a3]={a1, a2}

#в какие позиции входят v
#Ov[a1]={a1}
#Ov[a2]={a2}
#Ov[a3]={a3}

In [2]:
#количества состояний/входов/выходов
count = [3, 3, 3]
Sx = np.array([[1, 0, 1], [0, 0, 1], [0, 0, 0]])
Sv = np.array([[1, 0, 0], [0, 1, 0], [0, 0, 1]])

print("Матрица смежности:")
print("Sx = ")
print(Sx)
print("Sv = ")
print(Sv)

Матрица смежности:
Sx = 
[[1 0 1]
 [0 0 1]
 [0 0 0]]
Sv = 
[[1 0 0]
 [0 1 0]
 [0 0 1]]


In [3]:
def x1(x, v):
  return math.sin(math.log(abs(v[1] + v[0])) / x[2])

def x2(x, v):
  return math.cos(v[0] * x[2] - (x[1] / x[2])**3)

def x3(x, v):
  return math.sin(math.tan(v[0] / v[2]) - 3**(x[2]))

def v1(x, v):
  return (x[1] * v[0] * v[1] + math.tan(x[2])**2)

def v2(x, v):
  return (0.2 * v[0] + 33.0 * v[1] + v[2])

def v3(x, v):
  return (math.exp(x[0]) + .45 * math.log2(abs(x[1] * x[2])))

def y1(x, v):
  return (x[0] / v[0]) * math.sqrt(abs(x[0] / x[1]))

def y2(x, v):
  return math.sqrt(abs(math.sin(v[2])))

def y3(x, v):
  return v[2] * v[0] / v[0]

functions = [x1, x2, x3, v1, v2, v3, y1, y2, y3]

In [4]:
#Метод псевдообращения Бена-Израиля
def BenIsrael(A, eps = 0.000001): #А - np.array, eps - точность вычислительная
  I = np.diag(np.diag(np.ones([len(A),len(A)])))
  buf = 1
  i = 1

  if(np.linalg.norm(A, ord='fro') != 0): #если не нулевая норма
    gamma = 1.6 / (np.linalg.norm(A, ord='fro')**2)
    A1 = gamma * (A.T)
    i += 1
    A2 = A1.dot(2*I - A.dot(A1))
    E = np.linalg.norm(A2 - A1, ord=np.inf) / np.linalg.norm(A1, ord=np.inf)
  else: #если нулевая норма
    A1 = np.zeros((len(A[0]), len(A)))
    buf = 0

  while(E >= eps and buf):
    A1 = A2.copy()
    A2 = A1.dot(2*I - A.dot(A1))
    E = np.linalg.norm(A2 - A1, ord=np.inf) / np.linalg.norm(A1, ord=np.inf)
    i += 1

  return A1

#решение СЛАУ вида AX=B
def solvingSLAE(A, B):
  A = np.array(A.copy())

  #проверка на бесконечное количество решений
  A_psev = BenIsrael(A)

  X = A_psev.dot(B)
  return X

#=================================================================
#Генерация случайного вектора из 3-х элементов в диапазоне от -3 до 3
def generationRandomVector(elem = 3, start = -3, end = 3):
  vector = []
  for i in range(elem):
    vector.append(random.uniform(start, end))
  return vector

#=================================================================
#создание столбца таблицы
def createVariants(function, x, v, typeFunc = 1, elem = 100):
  vector = []

  if(typeFunc == 0):
    for i in range(elem):
      vector.append(function(generationRandomVector(), generationRandomVector()))
  else:
    for i in range(elem):
      vector.append(function(x[i], v[i]))

  return vector

#=================================================================
#Создание ДОМ в определённый момент времени
def createTableDNM(functions, count, step, oldTable = []): #count(x, v, y)
  table = []
  vector = []
  columns = createColumns(step, count)

  if(step == 0):
    for i in range(sum(count[0:2])):
      table.append(createVariants(functions[i], [], [], 0)) #xi, vi

  else:
    for i in range(count[0]): #xi
      table.append(createVariants(functions[i], (np.array(oldTable))[:, 0:count[0]], (np.array(oldTable))[:, count[0]:sum(count[0:2])]))
    for i in range(count[1]): #vi
      table.append(createVariants(functions[count[0] + i], [], [], 0))

  for i in range(count[2]): #yi
    table.append(createVariants(functions[sum(count[0:2]) + i], (np.array(table).T)[:, 0:count[0]], (np.array(table).T)[:, count[0]:sum(count[0:2])]))

  table = pd.DataFrame(np.array(table).T, columns = columns)
  return table

#заполнение
def createColumns(step, count):
  columns = []
  for i in range(count[0]):
    columns.append("x" + str(i + 1) + "(" + str(step) + ")")
  for i in range(count[1]):
    columns.append("v" + str(i + 1) + "(" + str(step) + ")")
  for i in range(count[2]):
    columns.append("y" + str(i + 1) + "(" + str(step) + ")")
  return columns

In [5]:
#functions = [x1, x2, x3, v1, v2, v3, y1, y2, y3]
model_0 = createTableDNM(functions, count, 0)
model_0

,x1(0),x2(0),x3(0),v1(0),v2(0),v3(0),y1(0),y2(0),y3(0)
0,0.674149,0.997277,0.014110,1.405620,53.100561,-0.242735,0.394329,0.490263,-0.242735
1,0.033119,-0.268519,0.482466,37.251401,-24.605685,0.073163,0.000312,0.270367,0.073163
2,-0.870074,-0.769218,-0.826778,0.919779,22.577057,0.682715,-1.006066,0.794293,0.682715
3,0.536880,-0.896459,-0.332681,47.892795,44.605224,11.292271,0.008675,0.977910,11.292271
4,0.142692,0.478275,0.980113,0.080225,61.981788,0.667861,0.971524,0.786961,0.667861
...,...,...,...,...,...,...,...,...,...
95,-0.457595,0.857643,0.297709,-3.328592,-80.601132,0.038614,0.100417,0.196480,0.038614
96,0.989244,-0.812944,-0.699202,0.585967,-16.068335,1.695722,1.862309,0.996096,1.695722
97,0.881270,0.783054,-0.999997,5.720153,13.037872,1.479385,0.163441,0.997910,1.479385
98,-0.974883,0.884237,-0.114296,-16.039962,75.072902,1.561999,0.063818,0.999981,1.561999


In [6]:
model_1 = createTableDNM(functions, count, 1, np.array(model_0))
model_1

,x1(1),x2(1),x3(1),v1(1),v2(1),v3(1),y1(1),y2(1),y3(1)
0,0.583862,0.187885,-0.461000,12.336899,73.115141,2.200118,0.083428,0.899126,2.200118
1,-0.854258,0.761849,-0.995892,3.066776,81.826355,2.519498,-0.294963,0.763373,2.519498
2,0.626210,0.004998,-0.753594,0.829364,-4.482186,-2.537800,8.451607,0.753504,-2.537800
3,-0.863330,-0.588363,0.954804,-1.653348,27.503885,-2.624822,0.632526,0.702905,-2.624822
4,-0.877347,0.999294,-0.321327,0.300136,80.629957,2.725920,-2.738998,0.635457,2.725920
...,...,...,...,...,...,...,...,...,...
95,0.736405,0.972779,-0.246228,12.043655,-0.867839,0.770604,0.053200,0.834607,0.770604
96,0.700955,-0.399185,-0.103682,6.583935,-84.173385,17.358420,0.141079,0.998413,17.358420
97,-0.208422,0.503454,0.524889,9.987367,-31.262373,0.532476,-0.013427,0.712508,0.532476
98,0.901275,0.996420,-0.906422,-0.375005,62.779213,1.141747,-2.285744,0.953605,1.141747


In [7]:
model_2 = createTableDNM(functions, count, 2, np.array(model_1))
model_2

,x1(2),x2(2),x3(2),v1(2),v2(2),v3(2),y1(2),y2(2),y3(2)
0,0.221847,0.787800,-0.986183,-0.897062,88.713590,12.486941,-0.131235,0.281685,12.486941
1,0.968247,-0.860220,0.694027,3.127800,33.404796,1.163117,0.328424,0.958146,1.163117
2,-0.989024,0.810961,-0.700373,13.061516,-46.470522,3.034773,-0.083621,0.326522,3.034773
3,-0.261608,0.224237,-0.849933,32.832748,55.546376,-0.109903,-0.008606,0.331183,-0.109903
4,-0.894311,0.135324,-0.558035,3.049957,-14.758029,1.249783,-0.753792,0.974123,1.249783
...,...,...,...,...,...,...,...,...,...
95,0.369197,-0.547162,-0.746157,16.666831,9.536508,-0.126041,0.018196,0.354552,-0.126041
96,0.903606,0.358751,-0.473929,-6.798879,43.722920,0.062219,-0.210928,0.249357,0.062219
97,-0.442233,-0.345290,-0.954559,0.439372,22.832957,0.396929,-1.139072,0.621762,0.396929
98,0.988470,-0.097384,-0.651999,-0.172500,-98.015758,2.175351,-18.256325,0.907059,2.175351


##1. Линейная форма функции пересчёта

###Для функции $x_1[t+1]$

In [9]:
#x3, v1, v2
x1_linear = model_0[['x3(0)', 'v1(0)', 'v2(0)']]
x1_linear['x1(1)'] = model_1['x1(1)']
x1_linear

<ipython-input-9-675b049a2243>:3: SettingWithCopyWarning: 
A value is trying to be set on a copy of a slice from a DataFrame.
Try using .loc[row_indexer,col_indexer] = value instead

See the caveats in the documentation: https://pandas.pydata.org/pandas-docs/stable/user_guide/indexing.html#returning-a-view-versus-a-copy
  x1_linear['x1(1)'] = model_1['x1(1)']


,x3(0),v1(0),v2(0),x1(1)
0,0.014110,1.405620,53.100561,0.583862
1,0.482466,37.251401,-24.605685,-0.854258
2,-0.826778,0.919779,22.577057,0.626210
3,-0.332681,47.892795,44.605224,-0.863330
4,0.980113,0.080225,61.981788,-0.877347
...,...,...,...,...
95,0.297709,-3.328592,-80.601132,0.736405
96,-0.699202,0.585967,-16.068335,0.700955
97,-0.999997,5.720153,13.037872,-0.208422
98,-0.114296,-16.039962,75.072902,0.901275


###Для функции $x_2[t+1]$

In [10]:
#x2, x3, v1
x2_linear = model_0[['x2(0)', 'x3(0)', 'v1(0)']]
x2_linear['x2(1)'] = model_1['x2(1)']
x2_linear

<ipython-input-10-2fee96f88008>:3: SettingWithCopyWarning: 
A value is trying to be set on a copy of a slice from a DataFrame.
Try using .loc[row_indexer,col_indexer] = value instead

See the caveats in the documentation: https://pandas.pydata.org/pandas-docs/stable/user_guide/indexing.html#returning-a-view-versus-a-copy
  x2_linear['x2(1)'] = model_1['x2(1)']


,x2(0),x3(0),v1(0),x2(1)
0,0.997277,0.014110,1.405620,0.187885
1,-0.268519,0.482466,37.251401,0.761849
2,-0.769218,-0.826778,0.919779,0.004998
3,-0.896459,-0.332681,47.892795,-0.588363
4,0.478275,0.980113,0.080225,0.999294
...,...,...,...,...
95,0.857643,0.297709,-3.328592,0.972779
96,-0.812944,-0.699202,0.585967,-0.399185
97,0.783054,-0.999997,5.720153,0.503454
98,0.884237,-0.114296,-16.039962,0.996420


###Для функции $x_3[t+1]$

In [11]:
#x3, v1, v3
x3_linear = model_0[['x3(0)', 'v1(0)', 'v3(0)']]
x3_linear['x3(1)'] = model_1['x3(1)']
x3_linear

<ipython-input-11-0bd7aeb4e4c6>:3: SettingWithCopyWarning: 
A value is trying to be set on a copy of a slice from a DataFrame.
Try using .loc[row_indexer,col_indexer] = value instead

See the caveats in the documentation: https://pandas.pydata.org/pandas-docs/stable/user_guide/indexing.html#returning-a-view-versus-a-copy
  x3_linear['x3(1)'] = model_1['x3(1)']


,x3(0),v1(0),v3(0),x3(1)
0,0.014110,1.405620,-0.242735,-0.461000
1,0.482466,37.251401,0.073163,-0.995892
2,-0.826778,0.919779,0.682715,-0.753594
3,-0.332681,47.892795,11.292271,0.954804
4,0.980113,0.080225,0.667861,-0.321327
...,...,...,...,...
95,0.297709,-3.328592,0.038614,-0.246228
96,-0.699202,0.585967,1.695722,-0.103682
97,-0.999997,5.720153,1.479385,0.524889
98,-0.114296,-16.039962,1.561999,-0.906422


###Для функции $y_1[t]$

In [13]:
#x1, x2, v1
y1_linear = model_0[['x1(0)', 'x2(0)', 'v1(0)']]
y1_linear['y1(0)'] = model_0['y1(0)']
y1_linear

<ipython-input-13-973dfb4ab1d4>:3: SettingWithCopyWarning: 
A value is trying to be set on a copy of a slice from a DataFrame.
Try using .loc[row_indexer,col_indexer] = value instead

See the caveats in the documentation: https://pandas.pydata.org/pandas-docs/stable/user_guide/indexing.html#returning-a-view-versus-a-copy
  y1_linear['y1(0)'] = model_0['y1(0)']


,x1(0),x2(0),v1(0),y1(0)
0,0.674149,0.997277,1.405620,0.394329
1,0.033119,-0.268519,37.251401,0.000312
2,-0.870074,-0.769218,0.919779,-1.006066
3,0.536880,-0.896459,47.892795,0.008675
4,0.142692,0.478275,0.080225,0.971524
...,...,...,...,...
95,-0.457595,0.857643,-3.328592,0.100417
96,0.989244,-0.812944,0.585967,1.862309
97,0.881270,0.783054,5.720153,0.163441
98,-0.974883,0.884237,-16.039962,0.063818


###Для функции $y_2[t]$

In [ ]:
#v3
y2_linear = model_0[['v3(0)']]
y2_linear['y2(0)'] = model_0['y2(0)']
y2_linear

###Для функции $y_3[t]$

In [14]:
#v1, v3
y3_linear = model_0[['v1(0)', 'v3(0)']]
y3_linear['y3(0)'] = model_0['y3(0)']
y3_linear

<ipython-input-14-3c69fe10d649>:3: SettingWithCopyWarning: 
A value is trying to be set on a copy of a slice from a DataFrame.
Try using .loc[row_indexer,col_indexer] = value instead

See the caveats in the documentation: https://pandas.pydata.org/pandas-docs/stable/user_guide/indexing.html#returning-a-view-versus-a-copy
  y3_linear['y3(0)'] = model_0['y3(0)']


,v1(0),v3(0),y3(0)
0,1.405620,-0.242735,-0.242735
1,37.251401,0.073163,0.073163
2,0.919779,0.682715,0.682715
3,47.892795,11.292271,11.292271
4,0.080225,0.667861,0.667861
...,...,...,...
95,-3.328592,0.038614,0.038614
96,0.585967,1.695722,1.695722
97,5.720153,1.479385,1.479385
98,-16.039962,1.561999,1.561999
